Install dependencies

In [12]:
import tensorflow as tf
import os

Couldnt get GPU to work, so I used CPU instead (I have a 2080ti, so I dont know what the problem is)

In [15]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Script to remove bad images

In [2]:
import imghdr as imghdr
import cv2
import os



image_exts = ['jpeg','jpg', 'bmp', 'png']
data_dir = 'data'
for classes in os.listdir(data_dir):
    for img in os.listdir(os.path.join(data_dir, classes)):
        
        try:
            if imghdr.what(os.path.join(data_dir, classes, img)) not in image_exts:
                print(os.path.join(data_dir, classes, img),"wrong extension")
                os.remove(os.path.join(data_dir, classes, img))
            img=cv2.imread(os.path.join(data_dir, classes, img))
        except:
            print((os.path.join(data_dir, classes, img)),"tought to open")
            os.remove(os.path.join(data_dir, classes, img))
            
        
            

data\happy\7-Habits-of-Happy-People-image.jpeg wrong extension
data\happy\Habits-of-Financially-Happy-People-1280x720.webp wrong extension
data\happy\happiness_thumbnail.jpg wrong extension
data\happy\Happy20People.jpg wrong extension
data\happy\traitshappypeople.jpg.webp wrong extension
data\sad\2_still_2x.gif wrong extension
data\sad\39843138-sad-man.jpg wrong extension
data\sad\crying-sad.gif wrong extension
data\sad\depositphotos_508407352-stock-photo-caucasian-male-sitting-floor-windows.jpg wrong extension
data\sad\very-sad-man-sitting-alone-on-white-background-depressed-young-man-sitting-businessman-vector.jpg wrong extension


Load Data